In [1]:
# Import PadreApp from a package
from padre.app import pypadre
# All interactions with padre should be done via the app

In [2]:
# List all available datasets(Toy Datasets)
print(pypadre.datasets.list())

In [3]:
# List all currently available local experiments
print(pypadre.experiments.list_experiments())

['Test Experiment SVM 021cc75d-', 'Test Experiment SVM 9617e580-', 'Decorated Experiment', 'Test Experiment SVM', 'Test Experiment SVM 45efad82-', 'Test Experiment SVM b797549a-', 'Test_Incorrect_Execute_Parameters', 'Test Experiment SVM c32e4875-', 'iPython Experiment1 ', 'KNN', 'SVC(Digits)', 'Regression using PCA and SVR(Digits)', 'Gaussian Process Regression', 'Grid search with Random Forest Classifier and PCA(Digits)', 'SVC(Iris)', 'IPython Experiment 3', 'Decision Tree(Iris)', 'Regression using PCA and SVR(Boston_House_Prices)', 'Decision Tree(Digits)', 'Test Experiment SVM bed4e8ac-', 'PLS Regression(Boston_House_Prices)', 'Grid search with Random Forest Classifier and PCA(Iris)', 'Torch', 'Linear SVR', 'Random Forest Classifier with PCA', 'Experiment using bagging classifier', 'Test Experiment SVM 4af9c9d7-', 'Test Experiment SVM 1da6639c-', 'PLS Regression(Diabetes)', 'Test_Incorrect_Estimator', 'KNN Regressor', 'Test Experiment SVM a367ba28-', 'Regression using PCA and SVR(Di

In [4]:
# Create a simple experiment via experiment creator utilizing a dataset displayed. Run it with multiple parameters
pypadre.experiment_creator.clear_experiments()
workflow = pypadre.experiment_creator.create_test_pipeline(['isomap embedding'])
pypadre.experiment_creator.create(name='iPython Experiment1 ', 
                                           description='Simple iPython notebook experiment',
                                           workflow=workflow, 
                                           dataset_list=['Digits'])
pypadre.experiment_creator.execute()

# Aimed at new users for running already created experiments or running from JSON files

INFO: 2019-03-11 13:21:10.869 <padre.experimentcreator.ExperimentCreator object at 0x7f32d00271d0>:	iPython Experiment1  created successfully!
{'Step_0': {'algorithm': {'attributes': {'path': '.steps[0][1]'},
                          'value': 'isomap embedding'},
            'doc': {'attributes': {'path': '.steps[0][1].__doc__'},
                    'value': 'Isomap Embedding\n'
                             '\n'
                             '    Non-linear dimensionality reduction through '
                             'Isometric Mapping\n'
                             '\n'
                             '    Read more in the :ref:`User Guide '
                             '<isomap>`.\n'
                             '\n'
                             '    Parameters\n'
                             '    ----------\n'
                             '    n_neighbors : integer\n'
                             '        number of neighbors to consider for each '
                             'poin

HTTPError: 403 Client Error:  for url: http://localhost:8080/api/datasets

In [5]:
# Using decorators to run experiments
from padre.ds_import import load_sklearn_toys
from padre import *
from padre.app import pypadre
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

@Workflow(exp_name="Decorated Experiment",
          description="Test experiment with decorators and SVC")
def create_test_pipeline():
    estimators = [('SVC', SVC(probability=True))]
    return Pipeline(estimators)


@Dataset(exp_name="Decorated Experiment")
def get_dataset():
    ds = [i for i in load_sklearn_toys()][2]
    return ds



In [6]:
ex = pypadre.experiments.run(decorated=True)
ex = run("Decorated Experiment")  # run the experiment and report
# Aimed at engineers and scientists for quick prototyping and trying out features

creating the workflow
creating the workflow


In [7]:
# Creating and running experiments via pure code (example.py)
def create_test_pipeline():
    from sklearn.pipeline import Pipeline
    from sklearn.svm import SVC
    from sklearn.decomposition import PCA
    # estimators = [('reduce_dim', PCA()), ('clf', SVC())]
    estimators = [('SVC', SVC(probability=True))]
    return Pipeline(estimators)



In [8]:
from padre.ds_import import load_sklearn_toys
from padre.core import Experiment
from padre.base import PadreLogger
from padre.eventhandler import add_logger

logger = PadreLogger()
logger.backend = pypadre.repository
add_logger(logger=logger)
ds = [i for i in load_sklearn_toys()][2]
ex = Experiment(name="Test Experiment SVM",
                    description="Testing Support Vector Machines via SKLearn Pipeline",
                    dataset=ds,
                    workflow=create_test_pipeline(), keep_splits=True, strategy="random")
ex.execute()

In [9]:
# The PyTorch wrapper supports the use of User Defined Callbacks 
from padre.core.wrappers.wrapper_pytorch import WrapperPytorch, CallBack
class TestCallbacks(CallBack):
    def on_compute_loss(self, loss):
        print('Function on compute loss. Loss = {loss}'.format(loss=loss))

    def on_epoch_end(self, obj):
        print('Epoch ended')

    def on_epoch_start(self, obj):
        print('Epoch started')

    def on_iteration_start(self, obj):
        print('Iteration started')

    def on_iteration_end(self, obj):
        print('Iteration ended')

In [10]:
# Running an experiment with a custom wrapper
from sklearn.pipeline import Pipeline
import os

print(os.getcwd())
# Define the parameters
params = dict()

import json
with open('../proof_of_concept/Pytorch/classification.json') as json_data:
    params = json.load(json_data)

obj = WrapperPytorch(params=params)
obj.set_callbacks([TestCallbacks()])
estimators = [('pytorch', obj)]
workflow = Pipeline(estimators)
ds = [i for i in load_sklearn_toys()][4]

ex = Experiment(name="Torch",
                description="Testing Torch via SKLearn Pipeline",
                dataset=ds,
                workflow=workflow)
ex.execute()

/home/christofer/PycharmProjects/PyPaDRe/tests/examples
Epoch started
Iteration started
Function on compute loss. Loss = 0.2704684294885457
1 0.2704684294885457
Iteration ended
Iteration started
Function on compute loss. Loss = 0.21455560341705018
2 0.21455560341705018
Iteration ended
Iteration started
Function on compute loss. Loss = 0.2263032243219632
3 0.2263032243219632
Iteration ended
Iteration started
Function on compute loss. Loss = 0.27795411551954824
4 0.27795411551954824
Iteration ended
Iteration started
Function on compute loss. Loss = 0.23827933860086242
5 0.23827933860086242
Iteration ended
Iteration started
Function on compute loss. Loss = 0.2274720181869051
6 0.2274720181869051
Iteration ended
Iteration started
Function on compute loss. Loss = 0.2731626186391353
7 0.2731626186391353
Iteration ended
Iteration started
Function on compute loss. Loss = 0.28381451386450396
8 0.28381451386450396
Iteration ended
Iteration started
Function on compute loss. Loss = 0.2740203875576

/home/christofer/PycharmProjects/test_folder/venv/lib/python3.6/site-packages/torch/nn/modules/container.py:91: UserWarning: Implicit dimension choice for softmin has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [11]:
# Compare the metrics between experiments and also between runs(Displaying pandas dataframe)
from padre.experimentexecutor import ExperimentExecutor
# Create an experiment with multiple runs
params_pca = {'num_components': [2, 3, 4, 5, 6]}
params_svr = {'C': [0.5, 1.0, 1.5],
              'poly_degree': [1, 2, 3]}
params_dict = {'SVR': params_svr, 'pca': params_pca}
workflow = pypadre.experiment_creator.create_test_pipeline(['pca', 'SVR'])
params_dict = pypadre.experiment_creator.convert_alternate_estimator_names(params_dict)
pypadre.experiment_creator.clear_experiments('IPython Experiment 3')
pypadre.experiment_creator.create(name='IPython Experiment 3',
                                  description='Experiment to test the comparison of metrics',
                                  dataset_list=['Boston_House_Prices'],
                                  workflow=workflow,
                                  params=params_dict
                                 )
experiments_executor = ExperimentExecutor(experiments=pypadre.experiment_creator.createExperimentList())
experiments_executor.execute(local_run=True, threads=1)
#from padre.metrics import CompareMetrics
#metrics = CompareMetrics(experiments_list=experiments_executor.experiments)
#print(metrics.show_metrics())
pypadre.metrics_evaluator.add_experiments(experiments_executor.experiments)
print(pypadre.metrics_evaluator.show_metrics())

Sequential Execution
{'Step_0': {'algorithm': {'attributes': {'path': '.steps[0][1]'},
                          'value': 'isomap embedding'},
            'doc': {'attributes': {'path': '.steps[0][1].__doc__'},
                    'value': 'Isomap Embedding\n'
                             '\n'
                             '    Non-linear dimensionality reduction through '
                             'Isometric Mapping\n'
                             '\n'
                             '    Read more in the :ref:`User Guide '
                             '<isomap>`.\n'
                             '\n'
                             '    Parameters\n'
                             '    ----------\n'
                             '    n_neighbors : integer\n'
                             '        number of neighbors to consider for each '
                             'point.\n'
                             '\n'
                             '    n_components : integer\n'
                      

                                     run  \
0   b843d613-c1f3-42d6-9fa4-2c0dda5e5242   
1   06bfb7b4-5186-499d-b62f-9a016372fd03   
2   7f8a031b-0f40-41bc-996c-6f21c2775201   
3   4ac73f8a-10dc-4856-ab96-41e8daf981ec   
4   b89136b4-ac5c-4304-9a73-86d0fb3e4ce4   
5   fcf5d767-1ddd-4df9-882c-d5bb506ff637   
6   343aefa9-f10e-4ae9-a212-831aad2f68f0   
7   4006ede4-ba63-49d1-8f7e-0b6947f14bd3   
8   c3887045-1170-4876-80c6-011bf4559d7f   
9   0501ea1b-15c6-476f-a837-de2a1c204ca7   
10  8ed5e0d9-c6ac-4e40-9888-946be6530ea3   
11  d56e3750-07d5-4af0-9ea8-6d2c2d205b95   
12  adb7df7f-3369-41ae-81c9-958ac306cc61   
13  ce95eb09-60ee-4b33-8ac6-b0c180f3076f   
14  34d08ebf-c9dd-4d91-a088-5e9aa2571360   
15  e1001e56-81e0-4da6-9570-bd8dce00f440   
16  e8d14d37-7595-4963-a457-c3b34b958f77   
17  71c2b3a5-80d0-437f-9b7a-5ff7c7779d25   
18  4ef84aa5-ec44-4b04-be44-4133279c1bb8   
19  5aa045fb-f2d1-46a3-9525-bc0932f47c96   
20  32f81a43-fd0b-4b6c-a4bb-5323f554519d   
21  f4c56617-28b6-4b4e-a984-e9bf

In [12]:
# Analyze metrics where the runs have specific values for a component
pypadre.metrics_evaluator.analyze_runs(
    ['principal component analysis.num_components.4', 'principal component analysis.num_components.5'])
df = pypadre.metrics_evaluator.display_results()
print(df)

                                     run  \
0   71c2b3a5-80d0-437f-9b7a-5ff7c7779d25   
1   ce95eb09-60ee-4b33-8ac6-b0c180f3076f   
2   4ac73f8a-10dc-4856-ab96-41e8daf981ec   
3   fe94fed3-212f-459c-a0c8-7924a43393ae   
4   ebefeb24-84ab-4ac5-8c6c-2a886f3c631a   
5   bd2507ae-3196-4b95-bb44-fb717b61072e   
6   adb7df7f-3369-41ae-81c9-958ac306cc61   
7   7f8a031b-0f40-41bc-996c-6f21c2775201   
8   26cfe4ba-18b7-46fb-9b0d-655cf7c4ce55   
9   a744fa05-fb17-4117-bf6d-931a279b0e19   
10  4ef84aa5-ec44-4b04-be44-4133279c1bb8   
11  0fcf4822-ae64-411b-8b93-3454e9603d0c   
12  4006ede4-ba63-49d1-8f7e-0b6947f14bd3   
13  03d67862-7b27-450f-b6e4-860747d40dfe   
14  c3887045-1170-4876-80c6-011bf4559d7f   
15  64c39c9d-90bd-4760-a7b6-3b0504156b84   
16  f3be079c-549e-4b80-83e3-d610123ca046   
17  46c5de59-6868-49d5-bb8f-8b1cc6e5c8ba   

                                   split                      dataset  \
0   3e92e0ff-e4c8-409e-b325-d86c5a63088a  Boston House Prices dataset   
1   15909f4c-a1c2

In [13]:
# Display only the metric specified by the user
pypadre.metrics_evaluator.analyze_runs(['principal component analysis.num_components.4'], ['mean_error'])
df = pypadre.metrics_evaluator.display_results()
print(df)

                                    run                                 split  \
0  71c2b3a5-80d0-437f-9b7a-5ff7c7779d25  3e92e0ff-e4c8-409e-b325-d86c5a63088a   
1  fe94fed3-212f-459c-a0c8-7924a43393ae  a31f788d-9c20-4611-8ca0-a85fb1f7e874   
2  adb7df7f-3369-41ae-81c9-958ac306cc61  b4666f6f-fa0c-4e1e-9384-663dc918bbe5   
3  7f8a031b-0f40-41bc-996c-6f21c2775201  7c783324-e3b2-41cc-be8c-0320a7ed4258   
4  26cfe4ba-18b7-46fb-9b0d-655cf7c4ce55  a29d88ca-8a2a-45fc-ace2-b36d1fc1f372   
5  4006ede4-ba63-49d1-8f7e-0b6947f14bd3  135b0ac8-2c2a-4651-89de-af7725243851   
6  03d67862-7b27-450f-b6e4-860747d40dfe  dfb25db8-7831-4a9d-8378-460e099adf0d   
7  f3be079c-549e-4b80-83e3-d610123ca046  cc1cfb6a-6ad0-4a13-a025-b5bda26b1e28   
8  46c5de59-6868-49d5-bb8f-8b1cc6e5c8ba  cc517561-4f00-4c83-a4b8-40a6d26b0b6a   

                       dataset epsilon-support vector regression.poly_degree  \
0  Boston House Prices dataset                                             1   
1  Boston House Prices datase

In [14]:
# Go online, register new user
# For registering, user has to use the browser to enter details

In [15]:
# Log in to server
# Implemented

In [16]:
# Upload/Sync experiments on demand
# Not Implemented. Experiments can only be uploaded during the execution of the experiment

In [17]:
# Look at the experiment via SPA and check the results and visualize them(if possible)

In [18]:
# Download an experiment from the server and run it.(via the experiment.JSON file) Example for reproducibility
# Not yet implemented

In [19]:
# Keep track of random seeds used in the experiment
# Not yet implemented

In [20]:
# Download a dataset from OpenML and upload it to the server
# Generate metrics for uploaded  datasets
# Currently in progress. Some datasets in OpenML do not work and raise exceptions

In [21]:
# Download a dataset from server
# Implemented

In [22]:
# Running an experiment with a dataset downloaded from the server and sync the experiment automatically
# Not implemented

In [23]:
# Running an experiment on the server cluster

In [24]:
# Tracking the progress of experiments including hyperparameter search
# Code to be included in the backend for logging the porogress

In [25]:
# Visualizing the results of an experiment/ dataset